In [1]:
def linkedin_resume_parser(path):

    import re
    import io
    import io
    from pdfminer.converter import TextConverter
    from pdfminer.pdfinterp import PDFPageInterpreter
    from pdfminer.pdfinterp import PDFResourceManager
    from pdfminer.pdfpage import PDFPage
    from docx2pdf import convert
    from pdfminer.layout import LAParams
    from pyresparser import ResumeParser
    from PyPDF2 import PdfFileWriter, PdfFileReader
    import pdfplumber
    
    import firebase_admin
    from firebase_admin import credentials
    from firebase_admin import firestore
    from firebase_admin import db
    

    #========================================Splitting the pdf-------
    pdf = pdfplumber.open(path)
    end = '"@@system.status.new"'
    inputpdf = PdfFileReader(open(path, "rb"))
    output = PdfFileWriter()
    endingPageList=[]
    pdfNameList=[]
    num=1
    for i in range(inputpdf.numPages):
        page = pdf.pages[i]
        text = page.extract_text()
        if text is not None:
            if end in text:
                with open("Resume%s.pdf" % num, "wb") as outputStream:
                    output.write(outputStream)
                    pdfNameList.append("Resume"+str(num)+".pdf")
                output = PdfFileWriter()
                num=num+1
                continue
            else:
                output.addPage(inputpdf.getPage(i))
    
    #========================================Function to extract raw text from the pdf-------
    def get_text_from_pdf(path):
        rsrcmgr = PDFResourceManager()
        retstr = io.StringIO()
        codec = 'utf-8'

        laparams = LAParams(line_overlap=.6, char_margin=1.5, line_margin=1.1, word_margin=0.3, boxes_flow=.6,
                             detect_vertical=False, all_texts=False)
        device = TextConverter(rsrcmgr, retstr, laparams=laparams)
        fp = open(path, 'rb')
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        password = ""
        maxpages = 0
        caching = True
        pagenos = set()
        for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password, caching=caching,
                                      check_extractable=True):
            interpreter.process_page(page)
        fp.close()
        device.close()
        string = retstr.getvalue()

        retstr.close()
        return string
    
        #====================================Cleaning the raw text============================
    canList=[]
    for resume in pdfNameList:
        read = get_text_from_pdf(resume)
        if os.path.getsize(resume)<1000:
            continue
        read = read.replace('\u25c6',' ')
        read = read.replace('➤➤',' ')
        read = read.replace('•',' ')
        read = read.replace('▶️',' ')
        read = read.replace('□',' ')
        read = read.replace('\x0c','')
        read = read.replace('\xa0','')

        read = read.replace('\n\n','\n')

        #========================================Gathering Skills information-------
        languages = {}
        applications = {}
        methods = {}
        delivery = {}
        impact ={}
        communication = {}

        lang = ['C#/.NET', 'C/C++', 'Go', 'Java', 'Javascript', 'Julia', 'MATLAB', 'PHP', 'R', 'Ruby', 'Scala', 'Visual Basic/VBA', 'Python', 'SQL', 'SAS']
        appli = ['Computer Vision', 'Advesarial Machine Learning','Machine Learning', 'Auto-ML', 'Basic Neural Networks', 'Neural Networks', 'Linear Regression', 'Logistic Regression', 'Regression','Convolutional Neural Networks', 'Generative Models', 'LSTMs', 'LSTM', 'Natural Language Processing', 'NLP', 'Predictive Models', 'Recurrent Neural Networks', 'Reinforcement Learning', 'Supervised Learning', 'Unsupervised Learning', 'Voice & Audio Processing']
        metho = ['Clustering','Data Analysis', 'Data Extraction', ' Data Compilation', 'Data Cleaning', 'Decision Analysis', 'Design of Experiments', 'Discrete Event Simulation', 'Game Theory', 'Linear & Integer Optimization', 'Multivariate Testing', 'Nonparametric Statistics', 'Parametric Statistics', 'Segmentation', 'Simulation', 'Monte Carlo', 'Stochastic Programming']
        deliv = ['Alteryx Analytics','Amazon Web Services','AWS', 'Anaconda','Apache Airflow','Apache Beam','Apache Hadoop','Hadoop','Hive','Apache Hive','Apache Spark','Apachec Kafka','Kafka','Caffe','Cassandra','Cloudera Workbench','Couchbase','Databricks','Dataiku','DataScience.com','Docker','Domino','Google Bigtable','Google Cloud Platform','GCP','H2O.ai','IBM Watson Studio','Keras','KNIME Analytics','Kubernetes/Kubeflow','MapReduce','MATLAB','MATLAB/Octave','Microsoft Azure','Azure','Microsoft R','MongoDB','MXnet','MySQL','Pandas','PostgreSQL','RapidMiner Studio','SAP Predictive Analytics','SAS Enterprise Miner', 'SAS EM', 'SAS', 'Tableau','TensorFlow','Teradata Analytics','TIBCO Data Science','Wolfram Mathematica']
        imp = ['Asset Utilization','Churn Prevention','Cost Optimization','Cross-/Upselling','Customer Analysis','Customer Experience','Debt Management','Demand Planning','E-Commerce','Fraud Prevention','Marketing Spend','Operational efficiencies','Predictive Maintenance','Pricing','Product Development','Promotion Optimization','Research & Development','Supply Chain Optimization','Workforce Planning']
        comm = ['Presentation Skills','Storytelling','Data Visualization','Writing Skills','Social Media Skills','Emotional Intelligence','Business Acumen']


        res=read.lower()
        skills=[]
        for i in lang:
            if res.find(' '+i.lower()+' ') != -1:
                languages[i] = 1


        for j in appli:
            if res.find(' '+j.lower()+' ') != -1:
                applications[j] = 1

        for k in metho:
            if res.find(' '+k.lower()+' ') != -1:
                methods[k] = 1

        for l in deliv:
            if res.find(' '+l.lower()+' ') != -1:
                delivery[l] = 1

        for m in imp:
            if res.find(' '+m.lower()+' ') != -1:
                impact[m] = 1

        for n in comm:
            if res.find(' '+n.lower()+' ') != -1:
                communication[n] = 1

        #========================================Gathering Education information-------

        splitted = read.split('\n')
        start=0
        for index,i in enumerate(splitted):
            if i =='Education':
                start=index+1

        eduList=splitted[start:-1]
        if len(eduList)%2!=0:
            eduList=eduList[:-1]
        realEduList=[]
        MasterList=['Masters','Master','M Sc ', 'M.Sc ','MS ','M S ','Master of Science','Master of Arts','MBA']
        BachelorList=['Bachelor','Bachelors', 'B Sc ', 'B.Sc ','BS ','B S ','Bachelor of Science ', 'Bachelor of Arts ','Associate of Science ','Associate ']
        DoctoralList=['Doctor of Philosophy','PhD']
        collegeList=['high school','university','college','academy','institute','international center']
        for index1,e in enumerate(eduList):
            if any(word.lower() in e.lower() for word in collegeList) and e!=eduList[-1]: 
                Start='N/A'
                End='N/A'
                Degree='N/A'
                Course='N/A'
                courseTerm = eduList[index1+1]
                if re.search(r'.*([1-3][0-9]{3})', eduList[index1+1]):
                    courseTerm = eduList[index1+1]
                elif e!=eduList[-2]:
                    if re.search(r'.*([1-3][0-9]{3})', eduList[index1+2]):
                        courseTerm = eduList[index1+1]+ ' '+eduList[index1+2]
                if any(word.lower() in courseTerm.lower() for word in MasterList):
                    Degree='Master'
                elif any(word.lower() in courseTerm.lower() for word in BachelorList):
                    Degree='Bachelor'
                elif any(word.lower() in courseTerm.lower() for word in DoctoralList):
                    Degree='Doctoral'
                if any(char.isdigit() for char in courseTerm):
                    firstDigit = re.search(r"\d", courseTerm).start()
                    eduDates=courseTerm[firstDigit:].strip().split('-')
                    Course=courseTerm[0:firstDigit].strip()[:-1]
                    if len(eduDates)>=1:
                        Start= eduDates[0]
                    if len(eduDates)>=2:
                        End= eduDates[1]
                else:
                    Course=courseTerm.strip()[:-1]
                realEduList.append({'Institution':e.strip(),
                                   'Course':Course,
                                    'Start':Start,
                                    'End':End,
                                    'Degree':Degree
                                    })
        #========================================Gathering Experience information-------
        name=splitted[0]
        location=splitted[1]
        expStartIndex=0
        expEndIndex=start
        for ind,e in enumerate(splitted):
            if e=='Experience':
                expStartIndex=ind+1
        experiences=splitted[expStartIndex:expEndIndex]

        jobs=[]
        experienceList=[]
        for i in range(len(experiences)):
            x=re.findall('\(.*?\)',experiences[i])
            if x and any(char.isdigit() for char in x[0]) and ('year' in x[0] or 'month' in x[0]) and ' at ' in experiences[i-1]:
                experienceList.append(i)
                startDate='N/A'
                endDate='N/A'
                dates=experiences[i].split('-')
                if len(dates)>=1:
                    startDate=dates[0].strip()
                if len(dates)>=2:
                    endDate=dates[1][:dates[1].find('(')].strip()
                month=0
                year=0
                temp = x[0][1:-1].split(" ")
                if temp[0].isnumeric(): 
                    if len(temp) > 2 and temp[2].isnumeric():
                        month+=int(temp[2])
                        year += int(temp[0])
                    elif(temp[1]=='year' or temp[1]=='years'):
                        year+=int(temp[0])
                    else:
                        month+=int(temp[0])
                    year += (month/12)
                    year = round(year,1)        
                comp=experiences[i-1].split(' at ')
                jobs.append({'Title':comp[0].strip(),
                             'Company':comp[1].strip(),
                            'Start':startDate,
                            'End':endDate,
                            'totalYearsInCompany':(str(year)+' years')})
        # extracting job description of all the jobs        
        for m in range(len(experienceList)):
            if experienceList[m]!=experienceList[-1]:
                if experienceList[m]+2==experienceList[m+1]:
                    jobs[m]['Description']='N/A'
                else:
                    description=''
                    for d in experiences[experienceList[m]+1:experienceList[m+1]-1]:
                        description=(description + ' '+ d.strip()).strip()
                    jobs[m]['Description']=description    
            else:
                if len(experiences)==experienceList[m]+1:
                    jobs[m]['Description']='N/A'
                else:
                    description=''
                    for d in experiences[experienceList[m]+1:]:
                        description=(description + ' '+ d.strip()).strip()
                    jobs[m]['Description']=description
        #================= Storing the data in firebase------------
        final_dict={'name':name,
                    'location':location,
                    'email':'N/A',
                    'phone':'N/A',
                    'background':{'career':jobs,
                                 'education':realEduList},
                   'skills':{
                       'applications':applications,
                       'langauges':languages,
                       'methods':methods,
                       'delivery':delivery,
                       'impact':impact,
                       'communication':communication
                   }}

        canList.append(final_dict)
        print(resume+" Done")
        
    mycred = credentials.Certificate('dev-key.json')

    try:
        firebase_admin.initialize_app(mycred)
    except ValueError:
        pass

    db = firestore.client()

    for c in canList:
        doc_ref = db.collection('users1').document()
        doc_ref.set(c)


In [11]:
#========================================Getting the pdfs from directory-------
import os
directory = r'C:\Users\sudar\Desktop\resume\ingested files\Test SAGs'
pathList=[]
for filename in os.listdir(directory):
    if filename.endswith(".pdf"):
        pathList.append(os.path.join(directory, filename))
    else:
        continue
for p in pathList:
    print(p)
    finalList=linkedin_resume_parser(p)
    print('File Done')

C:\Users\sudar\Desktop\resume\ingested files\Test SAGs\1. LI_Profile_Export_1.+Data+Scientist+-+D.C.+-+Top+Secret_20200714.pdf
Resume1.pdf Done
Resume2.pdf Done
Resume3.pdf Done
Resume4.pdf Done
Resume5.pdf Done
Resume6.pdf Done
Resume7.pdf Done
Resume8.pdf Done
File Done
C:\Users\sudar\Desktop\resume\ingested files\Test SAGs\2. LI_Profile_Export_SAGs+in+Data+Science_20200712.pdf
Resume1.pdf Done
Resume2.pdf Done
Resume3.pdf Done
Resume4.pdf Done
Resume5.pdf Done
Resume6.pdf Done
Resume7.pdf Done
Resume8.pdf Done
Resume9.pdf Done
Resume10.pdf Done
Resume11.pdf Done
Resume12.pdf Done
Resume13.pdf Done
Resume14.pdf Done
Resume15.pdf Done
Resume16.pdf Done
Resume17.pdf Done
Resume18.pdf Done
Resume19.pdf Done
Resume20.pdf Done
Resume21.pdf Done
Resume22.pdf Done
Resume23.pdf Done
Resume24.pdf Done
Resume25.pdf Done
File Done


In [6]:
filesize = os.path.getsize("Resume2.pdf")
type(filesize)

int

In [1]:
import re
import io
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage
from docx2pdf import convert
from pdfminer.layout import LAParams
from pyresparser import ResumeParser
from PyPDF2 import PdfFileWriter, PdfFileReader
import pdfplumber

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from firebase_admin import db
def get_text_from_pdf(path):
        rsrcmgr = PDFResourceManager()
        retstr = io.StringIO()
        codec = 'utf-8'

        laparams = LAParams(line_overlap=.6, char_margin=1.5, line_margin=1.1, word_margin=0.3, boxes_flow=.6,
                             detect_vertical=False, all_texts=False)
        device = TextConverter(rsrcmgr, retstr, laparams=laparams)
        fp = open(path, 'rb')
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        password = ""
        maxpages = 0
        caching = True
        pagenos = set()
        for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password, caching=caching,
                                      check_extractable=True):
            interpreter.process_page(page)
        fp.close()
        device.close()
        string = retstr.getvalue()

        retstr.close()
        return string
    
        #====================================Cleaning the raw text============================


In [2]:
canList=[]
# 'Resume1.pdf','Resume2.pdf','Resume3.pdf','Resume4.pdf','Resume5.pdf','Resume6.pdf','Resume7.pdf',
# 'Resume8.pdf','Resume9.pdf','Resume10.pdf',
#              'Resume11.pdf','Resume12.pdf',
pdfNameList=['Resume4.pdf','Resume5.pdf','Resume6.pdf','Resume7.pdf','Resume8.pdf','Resume9.pdf','Resume10.pdf',
             'Resume11.pdf','Resume12.pdf','Resume13.pdf','Resume14.pdf','Resume15.pdf','Resume16.pdf','Resume17.pdf',
             'Resume18.pdf','Resume19.pdf','Resume20.pdf','Resume21.pdf','Resume22.pdf','Resume23.pdf','Resume1.pdf',
            'Resume25.pdf','Resume26.pdf']
for resume in pdfNameList:
    read = get_text_from_pdf(resume)
    read = read.replace('\u25c6',' ')
    read = read.replace('➤➤',' ')
    read = read.replace('•',' ')
    read = read.replace('▶️',' ')
    read = read.replace('□',' ')
    read = read.replace('\x0c','')
    read = read.replace('\xa0','')

    read = read.replace('\n\n','\n')

    #========================================Gathering Skills information-------
    languages = {}
    applications = {}
    methods = {}
    delivery = {}
    impact ={}
    communication = {}

    lang = ['C#/.NET', 'C/C++', 'Go', 'Java', 'Javascript', 'Julia', 'MATLAB', 'PHP', 'R', 'Ruby', 'Scala', 'Visual Basic/VBA', 'Python', 'SQL', 'SAS']
    appli = ['Computer Vision', 'Advesarial Machine Learning','Machine Learning', 'Auto-ML', 'Basic Neural Networks', 'Neural Networks', 'Linear Regression', 'Logistic Regression', 'Regression','Convolutional Neural Networks', 'Generative Models', 'LSTMs', 'LSTM', 'Natural Language Processing', 'NLP', 'Predictive Models', 'Recurrent Neural Networks', 'Reinforcement Learning', 'Supervised Learning', 'Unsupervised Learning', 'Voice & Audio Processing']
    metho = ['Clustering','Data Analysis', 'Data Extraction', ' Data Compilation', 'Data Cleaning', 'Decision Analysis', 'Design of Experiments', 'Discrete Event Simulation', 'Game Theory', 'Linear & Integer Optimization', 'Multivariate Testing', 'Nonparametric Statistics', 'Parametric Statistics', 'Segmentation', 'Simulation', 'Monte Carlo', 'Stochastic Programming']
    deliv = ['Alteryx Analytics','Amazon Web Services','AWS', 'Anaconda','Apache Airflow','Apache Beam','Apache Hadoop','Hadoop','Hive','Apache Hive','Apache Spark','Apachec Kafka','Kafka','Caffe','Cassandra','Cloudera Workbench','Couchbase','Databricks','Dataiku','DataScience.com','Docker','Domino','Google Bigtable','Google Cloud Platform','GCP','H2O.ai','IBM Watson Studio','Keras','KNIME Analytics','Kubernetes/Kubeflow','MapReduce','MATLAB','MATLAB/Octave','Microsoft Azure','Azure','Microsoft R','MongoDB','MXnet','MySQL','Pandas','PostgreSQL','RapidMiner Studio','SAP Predictive Analytics','SAS Enterprise Miner', 'SAS EM', 'SAS', 'Tableau','TensorFlow','Teradata Analytics','TIBCO Data Science','Wolfram Mathematica']
    imp = ['Asset Utilization','Churn Prevention','Cost Optimization','Cross-/Upselling','Customer Analysis','Customer Experience','Debt Management','Demand Planning','E-Commerce','Fraud Prevention','Marketing Spend','Operational efficiencies','Predictive Maintenance','Pricing','Product Development','Promotion Optimization','Research & Development','Supply Chain Optimization','Workforce Planning']
    comm = ['Presentation Skills','Storytelling','Data Visualization','Writing Skills','Social Media Skills','Emotional Intelligence','Business Acumen']


    res=read.lower()
    skills=[]
    for i in lang:
        if res.find(' '+i.lower()+' ') != -1:
            languages[i] = 1


    for j in appli:
        if res.find(' '+j.lower()+' ') != -1:
            applications[j] = 1

    for k in metho:
        if res.find(' '+k.lower()+' ') != -1:
            methods[k] = 1

    for l in deliv:
        if res.find(' '+l.lower()+' ') != -1:
            delivery[l] = 1

    for m in imp:
        if res.find(' '+m.lower()+' ') != -1:
            impact[m] = 1

    for n in comm:
        if res.find(' '+n.lower()+' ') != -1:
            communication[n] = 1

    #========================================Gathering Education information-------

    splitted = read.split('\n')
    start=0
    for index,i in enumerate(splitted):
        if i =='Education':
            start=index+1

    eduList=splitted[start:-1]
    if len(eduList)%2!=0:
        eduList=eduList[:-1]
    realEduList=[]
    MasterList=['Masters','Master','M Sc ', 'M.Sc ','MS ','M S ','Master of Science','Master of Arts','MBA']
    BachelorList=['Bachelor','Bachelors', 'B Sc ', 'B.Sc ','BS ','B S ','Bachelor of Science ', 'Bachelor of Arts ','Associate of Science ','Associate ']
    DoctoralList=['Doctor of Philosophy','PhD']
    collegeList=['high school','university','college','academy','institute','international center']
    for index1,e in enumerate(eduList):
        if any(word.lower() in e.lower() for word in collegeList) and e!=eduList[-1]: 
            Start='N/A'
            End='N/A'
            Degree='N/A'
            Course='N/A'
            courseTerm = eduList[index1+1]
            if re.search(r'.*([1-3][0-9]{3})', eduList[index1+1]):
                courseTerm = eduList[index1+1]
            elif e!=eduList[-2]:
                if re.search(r'.*([1-3][0-9]{3})', eduList[index1+2]):
                    courseTerm = eduList[index1+1]+ ' '+eduList[index1+2]
            if any(word.lower() in courseTerm.lower() for word in MasterList):
                Degree='Master'
            elif any(word.lower() in courseTerm.lower() for word in BachelorList):
                Degree='Bachelor'
            elif any(word.lower() in courseTerm.lower() for word in DoctoralList):
                Degree='Doctoral'
            if any(char.isdigit() for char in courseTerm):
                firstDigit = re.search(r"\d", courseTerm).start()
                eduDates=courseTerm[firstDigit:].strip().split('-')
                Course=courseTerm[0:firstDigit].strip()[:-1]
                if len(eduDates)>=1:
                    Start= eduDates[0]
                if len(eduDates)>=2:
                    End= eduDates[1]
            else:
                Course=courseTerm.strip()[:-1]
            realEduList.append({'Institution':e.strip(),
                               'Course':Course,
                                'Start':Start,
                                'End':End,
                                'Degree':Degree
                                })
    #========================================Gathering Experience information-------
    name=splitted[0]
    location=splitted[1]
    expStartIndex=0
    expEndIndex=start
    for ind,e in enumerate(splitted):
        if e=='Experience':
            expStartIndex=ind+1
    experiences=splitted[expStartIndex:expEndIndex]

    jobs=[]
    experienceList=[]
    for i in range(len(experiences)):
        x=re.findall('\(.*?\)',experiences[i])
        if x and any(char.isdigit() for char in x[0]) and ('year' in x[0] or 'month' in x[0]) and ' at ' in experiences[i-1]:
            experienceList.append(i)
            startDate='N/A'
            endDate='N/A'
            dates=experiences[i].split('-')
            if len(dates)>=1:
                startDate=dates[0].strip()
            if len(dates)>=2:
                endDate=dates[1][:dates[1].find('(')].strip()
            month=0
            year=0
            temp = x[0][1:-1].split(" ")
            if temp[0].isnumeric(): 
                if len(temp) > 2 and temp[2].isnumeric():
                    month+=int(temp[2])
                    year += int(temp[0])
                elif(temp[1]=='year' or temp[1]=='years'):
                    year+=int(temp[0])
                else:
                    month+=int(temp[0])
                year += (month/12)
                year = round(year,1)        
            comp=experiences[i-1].split(' at ')
            jobs.append({'Title':comp[0].strip(),
                         'Company':comp[1].strip(),
                        'Start':startDate,
                        'End':endDate,
                        'totalYearsInCompany':(str(year)+' years')})
    # extracting job description of all the jobs        
    for m in range(len(experienceList)):
        if experienceList[m]!=experienceList[-1]:
            if experienceList[m]+2==experienceList[m+1]:
                jobs[m]['Description']='N/A'
            else:
                description=''
                for d in experiences[experienceList[m]+1:experienceList[m+1]-1]:
                    description=(description + ' '+ d.strip()).strip()
                jobs[m]['Description']=description    
        else:
            if len(experiences)==experienceList[m]+1:
                jobs[m]['Description']='N/A'
            else:
                description=''
                for d in experiences[experienceList[m]+1:]:
                    description=(description + ' '+ d.strip()).strip()
                jobs[m]['Description']=description
    #================= Storing the data in firebase------------
    final_dict={'name':name,
                'location':location,
                'email':'N/A',
                'phone':'N/A',
                'background':{'career':jobs,
                             'education':realEduList},
               'skills':{
                   'applications':applications,
                   'langauges':languages,
                   'methods':methods,
                   'delivery':delivery,
                   'impact':impact,
                   'communication':communication
               }}

    canList.append(final_dict)
    print(resume+" Done")

mycred = credentials.Certificate('dev-key.json')

try:
    firebase_admin.initialize_app(mycred)
except ValueError:
    pass

db = firestore.client()

for c in canList:
    doc_ref = db.collection('users1').document()
    doc_ref.set(c)

Resume4.pdf Done
Resume5.pdf Done
Resume6.pdf Done
Resume7.pdf Done
Resume8.pdf Done
Resume9.pdf Done
Resume10.pdf Done
Resume11.pdf Done
Resume12.pdf Done
Resume13.pdf Done
Resume14.pdf Done
Resume15.pdf Done
Resume16.pdf Done
Resume17.pdf Done
Resume18.pdf Done
Resume19.pdf Done
Resume20.pdf Done
Resume21.pdf Done
Resume22.pdf Done
Resume23.pdf Done
Resume1.pdf Done
Resume25.pdf Done
Resume26.pdf Done


Rough Work


In [12]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from firebase_admin import db

mycred = credentials.Certificate('dev-key.json')

try:
    firebase_admin.initialize_app(mycred)
except ValueError:
    pass

db = firestore.client()


all_docs = db.collection('users1').stream()

usersKeyList=[doc.to_dict() for doc in all_docs]

len(usersKeyList)

6031

In [1]:
import os

directory = r'C:\Users\sudar\Desktop\resume\TEST'
for filename in os.listdir(directory):
    if filename.endswith(".pdf"):
        print(os.path.join(directory, filename))
    else:
        continue

C:\Users\sudar\Desktop\resume\TEST\LI_Profile_Export_Search_20200725 (32).pdf
C:\Users\sudar\Desktop\resume\TEST\LI_Profile_Export_Search_20200725 (33).pdf
C:\Users\sudar\Desktop\resume\TEST\LI_Profile_Export_Search_20200725 (34).pdf
C:\Users\sudar\Desktop\resume\TEST\LI_Profile_Export_Search_20200725 (35).pdf
C:\Users\sudar\Desktop\resume\TEST\LI_Profile_Export_Search_20200725 (36).pdf
C:\Users\sudar\Desktop\resume\TEST\LI_Profile_Export_Search_20200725 (37).pdf
C:\Users\sudar\Desktop\resume\TEST\LI_Profile_Export_Search_20200725.pdf
